# Ebbflow Demo

Created By: Braeden Fieguth

Last Updated: 16/09/2024 

### 1. Install Ebbflow

Before starting, make sure you have `ebbflow` installed. If you haven't installed it yet, follow these steps:

1. **Open a Terminal**
2. **Activate your Python environment**: If you're using an environment manager like `conda`, make sure to activate your environment first. For example:
   ```bash
   conda activate my-environment-name
3. **Install ebbflow**: Now install the package by running:
   ```bash
   pip install ebbflow

## 2. Setup a Model Class

In `ebbflow`, models are defined by creating a new class that inherits from `BaseMechanisticModel`. This means your model class will automatically have access to methods for solving integrations and managing results.

To create your model, you need to define two methods:
1. `__init__`: This method is used to set up the initial conditions and parameters for your model.
2. `model`: This is where you define the actual differential equations that describe your model.

Let's create a simple class structure as a starting point:


In [2]:
import ebbflow  # First, import the package

# Tip: Name your class after the model you'll be creating
class CoriCycle(ebbflow.BaseMechanisticModel):  # Here we specify CoriCycle inherits BaseMechanisticModel
    def __init__(self):
        pass  # This is where we'll set up parameters for the model

    def model(self):
        pass  # This is where the equations for your model will go


## 3. Creating a Model

Now that we have the structure of our model defined, we can start adding the inputs and calculation steps.

### Defining the `__init__` Method:
In Python classes, the `__init__` method is called when a new object (or instance) of the class is created. This is where we define the parameters that can change each time we create a new instance of our model. Any constants used by your model should be defined here. We use `self` to make variables available throughout the class so that other methods can access them without needing to pass them around as arguments.

### Defining the `model` Method:
The `model` method describes the core behavior of our model, including how variables change over time. It must take two arguments:
1. `t`: The current time.
2. `state_vars`: A list of the current values of the state variables (variables that change over time).

In the `model` method, we define the differential equations and any intermediate steps (e.g., calculating concentrations or rates). After defining the calculations, we call `self.save()`, which stores the intermediate steps and results at that moment. Finally, the `model` method must return the differentials (rates of change) in the same order as the state variables.

Remember that `self` must be the first argument in every method of a class, as it refers to the instance of the class itself.


In [5]:
class CoriCycle(ebbflow.BaseMechanisticModel):  
    def __init__(self, kAB, kBO, YBAB, vol, outputs):
        # Assign model parameters to the instance (self)
        self.kAB = kAB        
        self.kBO = kBO
        self.YBAB = YBAB
        self.vol = vol        
        self.outputs = outputs  # List of variables to capture in the results
    
    def model(self, t, state_vars):
        # Get the current state variables (A and B)
        A = state_vars[0]
        B = state_vars[1]

        # Calculate intermediate values
        concA = A / self.vol
        concB = B / self.vol
        UAAB = self.kAB * concA  
        PBAB = UAAB * self.YBAB  
        UBBO = self.kBO * concB 

        # Define the differential equations (how A and B change over time)
        dAdt = -UAAB  
        dBdt = PBAB - UBBO

        # Capture the intermediate steps and results
        self.save()

        # Return the rates of change (differentials)
        return [dAdt, dBdt]
    

## 4. Using the Model

Now that we’ve defined our model, we can set some parameters and run it!

### Creating an Instance of the Model:
We start by creating an instance of our `CoriCycle` class. In Python, creating an instance means we’re making a specific version of our model with actual values for the constants. We don’t need to pass a value for `self` — Python automatically handles that for us. In this example, we’ll name the instance `model` and provide the necessary values for the model constants (`kAB`, `kBO`, `YBAB`, `vol`) and the list of variables we want to capture in the output.

### Running the Model:
Once the model instance is created, we can use the `run_model()` method to perform the integration (i.e., solve the differential equations over time). The `run_model()` method takes several important arguments:

- **equation**: Use "RK4". This specifies that we’ll be using the 4th-order Runge-Kutta method for solving the model.

- **`t_span`**: The time span over which to integrate (start_time, end_time).

- **`y0`**: The initial values for the state variables. These must be provided in the same order as they are returned in the `model()` method.

- **`t_eval`**: The time points where we want to evaluate the model (e.g., every 10 units of time).

- **`integ_interval`**: The step size for the integration. A smaller step size gives more accurate results but takes longer to calculate.

- **`name`**: Optional, a name for this particular model run. This allows us to save the results and refer to them by name later. By default the results are named result1, result2, etc.


In [7]:
import numpy as np

model = CoriCycle(
    kAB=0.42, kBO=0.03, YBAB=1.0, vol=1.0, 
    outputs=['t', 'A', 'B', 'concA', 'concB', 'dAdt']
)

model.run_model(
    "RK4", t_span=(0, 120), y0=[3.811, 4.473], t_eval=np.arange(0,121,10), 
    integ_interval=0.01, name="cori_1"
    )


Running Model...


## 5. Viewing the Results

Once the model has been run, it's often easiest to view the results in a table format. In `ebbflow`, the `BaseMechanisticModel` class provides a built-in method called `to_dataframe` to convert the results into a pandas `DataFrame`.

By default, `to_dataframe()` will return the results from the last model run. If you want to view results from a specific model run (if you’ve run the model multiple times), you can pass the `name` argument to specify which run to view.


In [8]:
result = model.to_dataframe()
display(result)


,t,A,B,concA,concB,dAdt
0,0.00,3.795027e+00,4.487629,3.795027e+00,4.487629,-1.593911e+00
1,9.99,5.714814e-02,6.292568,5.714814e-02,6.292568,-2.400222e-02
2,19.99,8.569694e-04,4.706319,8.569694e-04,4.706319,-3.599271e-04
3,29.99,1.285075e-05,3.487197,1.285075e-05,3.487197,-5.397315e-06
4,39.99,1.927044e-07,2.583389,1.927044e-07,2.583389,-8.093585e-08
5,49.99,2.889714e-09,1.913822,2.889714e-09,1.913822,-1.213680e-09
6,59.99,4.333292e-11,1.417794,4.333292e-11,1.417794,-1.819983e-11
7,69.99,6.498022e-13,1.050328,6.498022e-13,1.050328,-2.729169e-13
8,79.99,9.744159e-15,0.778102,9.744159e-15,0.778102,-4.092547e-15
9,89.99,1.461193e-16,0.576432,1.461193e-16,0.576432,-6.137010e-17


## 6. Continuing a Model Run

When using the "RK4" method for solving, it's possible to continue a model run from where it left off. To do this, we need to provide `prev_output`, which is a `DataFrame` containing the previous results. 

To continue the model:
1. Use `t_eval` and `t_span` to specify the new time interval for the continuation.
2. Set the initial state variables (`y0`) to the values they had at the end of the previous run.
3. All of this information (like the final state variables) can be gathered from the `DataFrame` we just created in the previous step.

By including the previous results in `prev_output`, the model knows where to continue.


In [12]:
# Get the new starting values for the state variables
new_stateVars =  result.iloc[-1, result.columns.isin(['A', 'B'])].tolist()

model = CoriCycle(
    kAB=0.5, kBO=0.03, YBAB=1.0, vol=1.0, 
    outputs=['t', 'A', 'B', 'concA', 'concB', 'dAdt']
)
model.run_model(
    "RK4", t_span=(120, 220), y0=new_stateVars, t_eval=np.arange(120,221,10), 
    integ_interval=0.01, prev_output=result
    )

new_result = model.to_dataframe()
display(new_result)


Running Model...


,t,A,B,concA,concB,dAdt
0,129.98,3.319897e-24,0.173618,3.319897e-24,0.173618,-1.659949e-24
1,139.98,2.236929e-26,0.128619,2.236929e-26,0.128619,-1.118465e-26
2,149.98,1.507231e-28,0.095284,1.507231e-28,0.095284,-7.536155e-29
3,159.98,1.015564e-30,0.070588,1.015564e-30,0.070588,-5.077821e-31
4,169.98,6.842818e-33,0.052293,6.842818e-33,0.052293,-3.421409e-33
5,179.98,4.610655e-35,0.038739,4.610655e-35,0.038739,-2.305327e-35
6,189.98,3.106635e-37,0.028699,3.106635e-37,0.028699,-1.553317e-37
7,199.98,2.093234e-39,0.021261,2.093234e-39,0.021261,-1.046617e-39
8,209.98,1.410410e-41,0.015750,1.410410e-41,0.015750,-7.052050e-42
9,219.98,9.503268e-44,0.011668,9.503268e-44,0.011668,-4.751634e-44


## 7. Common Issues

When using `ebbflow`, it’s important to define your model class in a specific way. 

### Renaming `model`:
In the example below, the class `MissingModelMethod` is missing the correct `model` method. Instead, the method has been named `cori_cycle_model`. This will result in a `TypeError` when trying to create an instance of the class. `ebbflow` expects your model method to be named exactly `model`, as this is the method that the solver will call during integration.

This issue is easy to fix: just ensure that your method is named `model` and follows the correct method signature.

### Example:
In the code below, we have mistakenly renamed the `model` method:

In [16]:
import ebbflow

class MissingModelMethod(ebbflow.BaseMechanisticModel):
    def __init__(self, kAB, kBO, YBAB, vol, outputs):
        self.kAB = kAB        
        self.kBO = kBO
        self.YBAB = YBAB
        self.vol = vol        
        self.outputs = outputs
    
    def cori_cycle_model(self, t, state_vars):  # NOTE We changed the method name 
        A = state_vars[0]
        B = state_vars[1]

        concA = A / self.vol
        concB = B / self.vol
        UAAB = self.kAB * concA  
        PBAB = UAAB * self.YBAB  
        UBBO = self.kBO * concB 

        dAdt = -UAAB  
        dBdt = PBAB - UBBO

        self.save()
        return [dAdt, dBdt]
    
model = MissingModelMethod(
    kAB=0.5, kBO=0.03, YBAB=1.0, vol=1.0,   
    outputs=['t', 'A', 'B', 'concA', 'concB', 'dAdt']
)
        

TypeError: Can't instantiate abstract class MissingModelMethod without an implementation for abstract method 'model'

### Forgetting `self.save()`

In `ebbflow`, the `self.save()` method is essential for capturing intermediate results during the model run. If you forget to include `self.save()` in your `model` method, or if you comment it out, a `ValueError` will be raised. This ensures that your model is capturing the necessary outputs during each step of the integration.

### Example:

In the code below, we have removed the `self.save()` call, which will cause an error when running the model. 

This issue is easy to fix: simply add the `self.save()` method back into the `model` method.


In [24]:
import ebbflow

class MissingModelMethod(ebbflow.BaseMechanisticModel):
    def __init__(self, kAB, kBO, YBAB, vol, outputs):
        self.kAB = kAB        
        self.kBO = kBO
        self.YBAB = YBAB
        self.vol = vol        
        self.outputs = outputs
    
    def model(self, t, state_vars):  
        A = state_vars[0]
        B = state_vars[1]

        concA = A / self.vol
        concB = B / self.vol
        UAAB = self.kAB * concA  
        PBAB = UAAB * self.YBAB  
        UBBO = self.kBO * concB 

        dAdt = -UAAB  
        dBdt = PBAB - UBBO

        # NOTE self.save has been removed
        return [dAdt, dBdt]
    
model = MissingModelMethod(
    kAB=0.5, kBO=0.03, YBAB=1.0, vol=1.0,   
    outputs=['t', 'A', 'B', 'concA', 'concB', 'dAdt']
)

ValueError: The method `self.save()` is not called in the `model` method.